## Getting started

For those of you who haven't used notebooks like this before, they provide a convenient way to do exploratory data science work in a format that combines your code and its output in an easy-to-share way. Above, you will see a dropdown menu that allows you to switch a cell in the notebook between code and markdown text (like here).

To create a new cell below your current position, press the + button above. To run the code in a cell, or render your markdown text, press Shift+Enter.

To begin, we import a couple python libraries we'll need throughout the exercise. Run this cell:

In [ ]:
import pandas as pd
import numpy as np

### Read in the data using pd.read_csv(), supplying the filename "tweets.csv".

Then, inspect the first few rows of your data frame using the .head() method, supplying n=10 as an argument to show the top 10 rows.

In [ ]:
tweets = 

### There's a problem -- we have some missing data. Let's remove lines in our data that don't contain a tweet.

Missing data will cause a future step to fail, when we go to process the text of these tweets.

Pandas makes this easy with the notnull() function. Take note of how Pandas makes use of square brackets to identify an individual column of that data frame.

Then, verify that the missing rows have been properly excluded.

In [ ]:
tweets = tweets[pd.notnull(tweets['tweet_text'])]

### There are three columns of data here, but for now let's limit ourselves to tweet_text and tweet_sentiment. 

Assign each of these columns to new variables called "text" and "labels".

In [ ]:
text = 
labels = 

### Turning our text into numbers

Regardless of the data you're using, machine learning is always a mathematical task. That means we have to convert our raw text into a usable format -- in this case, counts of word occurrences. This is the information that will allow us to train a model and use it to make predictions. This next step also makes our first use of the scikit-learn ML library.

Take the CountVectorizer instance created below and apply it to the text of your tweets using the .fit() method.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()


### What does the fitted CountVectorizer object look like?

CountVectorizer produces a __matrix__ where the rows are tweets, and the columns represent every word in our "vocabulary" (i.e., all the words in our whole set of tweets). The first step in generating the matrix is to figure out its dimensions. The vocabulary&#95; object created by the .fit() method is a mapping from vocabulary words to matrix columns:

In [ ]:
print(count_vect.vocabulary_)

Use the .get() method and look up some words to see if they're in our vocabulary or not (if they are, .get() will return a number; if they aren't, it will return nothing). Notice anything interesting or surprising? Why is there an entry for "iphone", but not "iPhone"?

Now that we've generated a vocabulary from our data, use the .transform() method, supplying our text as an argument, to actually create the document-term matrix that contains the counts for the words in each tweet. 

In [ ]:
counts = 

# Pause here for a moment before continuing on to modeling

If you've finished early, consider playing around some more with your document-term matrix, or look at the CountVectorizer documentation to learn about options you can play with later: 

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

After the group members have their data featurized, we'll talk for a bit about the machine learning approach we'll use in this lab: a Naive Bayes classifier.

The math involved in Naive Bayes is a bit complex, but scikit-learn makes it easy to train and use a model:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

### Training and playing around with your model

Now that we've imported MultinomialNB (a version of the classifier that can handle more than 2 labels), now we need to train it. This follows a two-step process very similar to what we did with CountVectorizer above: create a model instance, and then train it using the .fit() method. The only difference is that here, fit() will take TWO arguments -- counts and labels, in that order. 

In [ ]:
model1 = MultinomialNB()


Now that you've got your model trained, try evaluating its predictions on some sentences. What does it get right? Can you trick it into being wrong? How?

Here's a sample sentence to get you started. Just modify the sample sentence however you want:

In [ ]:
print(model1.predict(count_vect.transform(["I love my iPhone!!"])))

### Measuring model accuracy

Providing the model with sample sentences is fun, but it's not the best way to actually measure its performance. We need a large set of labeled data to really see how good the predictions are!

Let's start by seeing how many of our tweets the model classifies correctly. Here, we first generate predictions using our model, then sum up how many times the predictions match the actual labels. We divide this number by the total number of items to get a percentage:

In [ ]:
predictions = model1.predict(counts)
sum(predictions == labels)/float(len(labels))

### Train-test splits

You probably got about 80% accuracy from that comparison. Not bad! -- unfortunately, though, we can't trust that number. We can't evaluate a model using the data it was trained on -- that'd be cheating, because it already knows the answers to those items!

As a first step in addressing this problem, let's take our data and divide it up into two parts, with about 80% of it for training our model and 20% for testing with. We have about 9000 rows of data to work with here, so let's train on the first 7500, and test on the rest. The code to create these subsets is included below for convenience; run it and then train and evaluate your model in the same way we did above:
* Train the model using counts_train and labels_train
* Generate predictions on counts_test
* Compare those predictions to labels_test

In [ ]:
counts_train = counts[0:7500]
counts_test = counts[7500:]
labels_train = labels[0:7500]
labels_test = labels[7500:]

In [ ]:
model2 = 

In [ ]:
predictions2 = 

### Baselines

Wow! Not as good as we thought, huh? Is our model even any good at all? One way to answer that is by comparing our accuracy to a "baseline", i.e. a simple model that makes dumb (but still principled) predictions. One popular baseline measure would be to just predict the most common label for everything -- let's try that and compare. Take the dummy model instantiated below; train and evaluate it in the same way you did above.

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')

In [ ]:
dummy_predictions = 

# Pause here for a moment before continuing on to further evaluation.

If you've finished this section early, feel free to play around with other train-test splits. Do you get different results? What does this fact suggest about the best way to evaluate your model?

### Cross-validation

Let's cross-validate a classifier on our data and see how it holds up.

Below, create a new MultinomialNB instance, and then evaluate it using the cross_val_score() function. cross_val_score takes __4__ arguments: the model name, our "counts" and "labels" objects, and the number of iterations (or "folds") to use in the evaluation, in that order. The default number of folds is 3, but let's do 10 -- expressed as cv=10.

Assign the result of cross_val_score to the variable "scores", and print it. Then print its average, obtained via the .mean() method.

In [ ]:
from sklearn.model_selection import cross_val_score
model3 = 
scores = 

Here, we got a bit lucky -- our average cross-validation accuracy is about the same as our accuracy on the earlier test set. But it should still be clear why it's important to do this -- accuracy on individual folds varied by almost 10%!

### The Confusion Matrix

As a first step towards understanding how to improve our model, let's take a closer look at what it's getting wrong. To do this, we look at the __confusion matrix__, which matches up the labels the model predicted against the labels things actually had. Here, we stick with our previous train-test split for simplicity.

Run this code:

In [ ]:
from sklearn.metrics import confusion_matrix
import pylab as pl

sentiment_labels = ['positive', 'neutral', 'negative']
cm = confusion_matrix(labels_test, predictions2, sentiment_labels)
print(cm)
fig = pl.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
pl.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + sentiment_labels)
ax.set_yticklabels([''] + sentiment_labels)
pl.xlabel('Predicted')
pl.ylabel('True')
pl.show()

The confusion matrix is displayed in two ways: graphically, and as a less intuitive numerical matrix right above the graphical version. Here are some things to look for:

* Which label gets predicted correctly most often?
* What types of errors are the most common?
* Are the different labels in our data equally well-represented?

The confusion matrix reveals a big issue with our data, and a big issue in machine learning in general: __class imbalance__. We have ___way___ more Neutral items than anything else, and very few Negative items at all! It should be clear that better-represented labels are captured better by the model, and that less well-represented items frequently get misclassified with one of the more common labels.

### Get more data!

Fortunately, we happen to have some additional negative tweets lying around. They're not from exactly the same dataset, but a pretty similar one so maybe they'll help out. Read in the negative_tweet_supplement.csv file as a new data frame, split out the columns, and add them to our existing "text" and "labels" objects using the .append() method.

In [ ]:
new_negs = 

In [ ]:
new_neg_text = 
new_neg_labels = 
combined_text = 
combined_labels = 

Now, because we have changed our original text object, we need to re-fit it with CountVectorizer and transform our combined text into a new document-term matrix, as we did earlier. Do that now (you can go ahead and overwrite the existing count_vect object if you like), assigning the new document-term matrix to the variable "combined_counts".

In [ ]:
combined_counts = 

Now that we have added some more negative items to our data, let's redo our cross-validation with a new model, making sure to update the arguments in cross_val_score() to reflect the new variable names for our longer counts and labels objects.

In [ ]:
model4 = 
new_scores = 

### Final bits: pipelines

The last feature of scikit-learn we want to touch on, and a powerful resource for machine learning in general, is the pipeline. This does not introduce new content as much as it presents a way of chaining together the various bits of data preparation and modeling we have done so far. This is really useful for iterating on your model development, and also highlights a great feature of scikit-learn which is the consistency of object types and methods shared across its various core functions.

A couple new options are introduced here as well, without evaluation, as a way of getting you started on future work tuning your model. One of these is the ngram_count option within CountVectorizer -- this lets you include not just individual words, but word _sequences_, as features. That can be valuable -- think of the difference between "Great!" and "Oh, great!" (or even "I like" and "I don't like").

The other is a feature selection step that comes before model training. Currently, every word in our vocabulary is considered as informative when training the model. That seems fair, but is it the best thing to do? What about words that only occur once in the data? There's only so much probability to go around, and the often very large number of these rare features can diminish the power of other more important ones. Feature selection also helps to combat __overfitting__, where the model learns its training data __too__ well, and has trouble generalizing. (Of course, being too aggressive can yield the opposite problem of underfitting as well -- you wouldn't want to train a model on just one feature out of thousands!)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

p = Pipeline(steps=[('counts', CountVectorizer(ngram_range=(1, 2))),
                ('feature_selection', SelectKBest(chi2, k=10000)),
                ('multinomialnb', MultinomialNB())])
p.fit(combined_text, combined_labels)

Once you've fit a model using a pipeline, to generate predictions just pass it the text object itself -- no need to pass it the counts directly as before, because CountVectorizer is one of the steps!

Of course, because we just trained this model on all our tweets, we don't actually have anything new to apply it to. Which leads us to...

# Now for the challenge: extending your model

When we do machine learning work, we don't just build models that exist in a vacuum. They're applied to new data all the time! One of the big challenges there is that the new data doesn't always look like the data the model was trained on, and we have to periodically re-train the model to account for this.

Let's start to work with some new data. In your project directory, you'll see a file called holdout_set.csv -- __don't inspect it!__. One of the classic features of machine learning problems is that there's a dataset you use to evaluate your model, but you can never know exactly what it contains (otherwise, you could cheat! -- not that any of you would ;)

You can know this much about the contents of holdout.csv, though:
* The column names are the same as we've been using -- "tweet_text" and "tweet_sentiment".
* The sentiment labels are the same as well: "positive", "negative", "neutral".
* The holdout set is composed, to varying degrees, of tweets taken from two other datasets. The balance of these datasets -- one about air travel and one about a political debate -- have been included for you to work with.

Your goal is to build the model that achieves the best accuracy on this holdout set.

Possible strategies include combinations of the following:
* Incorporating data from the (non-holdout!) air travel and debate datasets
* Balancing your training data in different ways
* Trying out a different class of model, like logistic regression or random forest (more info on these and others here: http://scikit-learn.org/stable/modules/multiclass.html)
* Experimenting with different model parameters or feature selection: (try setting ngram_range to include 2 or 3 word sequences! try removing stopwords (sample list below)! try not lower-casing your text! try limiting your model to only a certain number of features!)

No team will have time to try every possible combination of strategies here. It's a safe bet that the winner will, to a degree at least, pull in some new training data, but beyond that just have fun and let's see what you come up with!

As you work on improving your model, try not to change too many things at once. Try to keep track of what things worked, what things didn't work, and what made the biggest difference!

You may wish to wrap your model training and evaluation steps up in one or more functions so you can iterate more efficiently. That's fine.

In [ ]:
## Just in case you want to explore using them
stopwords = ['i', 'you', 'me', 'they', 'is', 'are', 'to', 'the', 'a', 'an', 'at', 'were', 'of', 'at']

In [ ]:
holdout = pd.read_csv("holdout_set.csv") ## No peeking!